# Chemical equilibrium with fixed fugacity

<p class="acknowledgement">Written by Allan Leal (ETH Zurich) on Jan 7th, 2022</p>

This tutorial demonstrates the calculation of calcite (CaCO{{_3}}) solubility
in a saline aqueous solution for given fugacity of CO{{_2}}. 

The code block below produces a chemical system containing an aqueous phase (to
model our saline aqueous solution) and a solid mineral phase (to model our
calcite mineral). 

In [47]:
import reaktoro as rkt

db = rkt.PhreeqcDatabase("phreeqc.dat")

solution = rkt.AqueousPhase(rkt.speciate("H O Na Cl C Ca"))
solution.setActivityModel(rkt.ActivityModelHKF())

calcite = rkt.MineralPhase("Calcite")

system = rkt.ChemicalSystem(db, solution, calcite)

In our desired chemical equilibrium calculation, the following properties are
constrained at equilibrium:

* temperature;
* pressure; and
* fugacity of CO{{_2}};

The next step is to create an object of class {{EquilibriumSpecs}} containing the
above constraint specifications for the chemical equilibrium calculation we
want to perform.

In [48]:
specs = rkt.EquilibriumSpecs(system)
specs.temperature()
specs.pressure()
specs.fugacity("CO2(g)")

```{note}
Because the fugacity of CO{{_2}} is constrained, the chemical equilibrium
problem will presume that the system is open to CO{{_2}}. This means that
during the calculation, enough CO{{_2}} is allowed to leave or enter the system
so that the fugacity constraint is satisfied. 
```

Once we have created this specifications object, we can now create our
dedicated and optimized equilibrium solver to solve equilibrium problems
with such constraints, which is done next:

In [49]:
solver = rkt.EquilibriumSolver(specs)

Let's now create an initial chemical state for our chemical system, one that
represents a 1 molal NaCl aqueous solution mixed with 10 g of calcite mineral.
We do this next by creating an object of class {{ChemicalState}}:

In [50]:
state = rkt.ChemicalState(system)
state.temperature(50.0, "celsius")
state.pressure(10.0, "bar")
state.set("H2O", 1.0, "kg")
state.set("Na+", 1.0, "mol")
state.set("Cl-", 1.0, "mol")
state.set("Calcite", 10, "g")

```{note} 
This initial chemical state is not yet in chemical equilibrium! The `state` object created above will be equilibrated later using `solver`.
```

Let's now specify the temperature, pressure, and fugacity conditions (their
actual values) we want to impose at equilibrium. We do this by using an object
of class {{EquilibriumConditions}}:

In [51]:
fCO2g = 0.1  # 0.1 bar

conditions = rkt.EquilibriumConditions(specs)
conditions.temperature(50.0, "celsius")
conditions.pressure(10.0, "bar")
conditions.fugacity("CO2(g)", fCO2g, "bar")

We have everything we need now to perform the chemical equilibrium calculation:

* an equilibrium solver (the `solver` object of class {{EquilibriumSolver}});
* a chemical state to equilibrate (the `state` object of class {{ChemicalState}}); and
* the conditions we are imposing for this equilibrium state (the `conditions`
  object of class {{EquilibriumConditions}}).

The code below will perform the equilibrium calculation and when it is
finished, our `state` object will be in a chemical equilibrium state:

In [52]:
result = solver.solve(state, conditions)

It's always advisable to verify if the calculation succeeded:

In [53]:
print("Successful computation!" if result.optima.succeeded else "Computation has failed!")

Successful computation!


Let's check the computed chemical equilibrium state: 

In [54]:
print(state)

+-----------------+-------------+------+
| Property        |       Value | Unit |
+-----------------+-------------+------+
| Temperature     |      323.15 |    K |
| Pressure        |       1e+06 |   Pa |
| Element Amount: |             |      |
| :: H            |     111.012 |  mol |
| :: C            |    0.106923 |  mol |
| :: O            |       55.82 |  mol |
| :: Na           |           1 |  mol |
| :: Cl           |           1 |  mol |
| :: Ca           |    0.099909 |  mol |
| Species Amount: |             |      |
| :: CO3-2        |   1.561e-05 |  mol |
| :: H+           | 2.52434e-07 |  mol |
| :: H2O          |     55.5007 |  mol |
| :: CO2          |  0.00152145 |  mol |
| :: (CO2)2       |       1e-16 |  mol |
| :: HCO3-        |  0.00929598 |  mol |
| :: CH4          |       1e-16 |  mol |
| :: Ca+2         |  0.00552591 |  mol |
| :: CaCO3        | 5.70421e-06 |  mol |
| :: CaHCO3+      | 0.000132019 |  mol |
| :: CaOH+        | 1.35322e-09 |  mol |
| :: Cl-        

And this is it. The above table presents the computed chemical state at
equilibrium for the entire chemical system (aqueous solution and mineral
calcite). Note that calcite was not fully dissolved with the specified
conditions, and the solution is thus saturated with it. If this was not the
case, we should then have increased the initial amount of calcite (e.g., 100 g
instead of 10 g), since we are interested in computing the solubility of
calcite.

Let's now have a more in-depth look at the chemical state of the system by
creating an object of class {{ChemicalProps}} and printing it:

In [55]:
props = rkt.ChemicalProps(state)
print(props)

+----------------------------------------+--------------+-----------+
| Property                               |        Value |      Unit |
+----------------------------------------+--------------+-----------+
| Temperature                            |       323.15 |         K |
| Pressure                               |        1e+06 |        Pa |
| Volume                                 |   0.00103295 |        m3 |
| Gibbs Energy                           |     -5891.62 |         J |
| Enthalpy                               |      1596.71 |         J |
| Entropy                                |      23.1729 |       J/K |
| Internal Energy                        |      563.759 |         J |
| Helmholtz Energy                       |     -6924.57 |         J |
| Element Amount:                        |              |           |
| :: H                                   |      111.012 |       mol |
| :: C                                   |     0.106923 |       mol |
| :: O              

This table is long but gives you a complete picture of all the system's
important chemical and thermodynamic properties at the computed chemical
equilibrium state. This table shows, for example, the activities of the
species, their chemical potentials, and standard thermodynamic properties.

For properties more pertinent to aqueous solutions, we can create an object of
class {{AqueousProps}}:

In [56]:
aprops = rkt.AqueousProps(state)
print(aprops)

+--------------------------+-------------+-------+
| Property                 |       Value |  Unit |
+--------------------------+-------------+-------+
| Temperature              |      323.15 |     K |
| Pressure                 |       1e+06 |    Pa |
| Ionic Strength (Effect.) |     1.01516 | molal |
| Ionic Strength (Stoich.) |     1.01674 | molal |
| pH                       |     6.79516 |       |
| pE                       |     2.50064 |       |
| Eh                       |    0.160341 |     V |
| Element Molality:        |             |       |
| :: C                     |   0.0126796 | molal |
| :: Na                    |     1.00014 | molal |
| :: Cl                    |     1.00014 | molal |
| :: Ca                    |  0.00566443 | molal |
| Species Molality:        |             |       |
| :: CO3-2                 | 1.56122e-05 | molal |
| :: H+                    | 2.52469e-07 | molal |
| :: CO2                   |  0.00152166 | molal |
| :: (CO2)2                | 1.

Let's now find out if our computed chemical equilibrium state satisfies the imposed fugacity constraint. If this is the case, the following must be observed:

$$\mu_{\mathrm{CO_{2}(aq)}}=\mu_{\mathrm{CO_{2}(g)}},$$

where

$$\mu_{\mathrm{CO_{2}(g)}}\equiv\mu_{\mathrm{CO_{2}(g)}}^{\circ}+RT\log f_{\mathrm{CO_{2}(g)}}.$$

For the quantities above, note that:

* $\mu_{\mathrm{CO_{2}(aq)}}$, the chemical potential of aqueous species
  CO<sub>2</sub>, can be obtained from the {{ChemicalProps}} object;
* $\mu_{\mathrm{CO_{2}(g)}}^{\circ}$, the standard chemical potential of the
  gaseous species CO<sub>2</sub>(g), cannot be obtained from the
  {{ChemicalProps}} object because CO<sub>2</sub>(g) is not considered in the
  chemical system. For this property, we can compute it at specified
  temperature and pressure conditions as shown next;
* $f_{\mathrm{CO_{2}(g)}}$ this is the imposed fugacity value for CO<sub>2</sub>(g).

The code below verifies the above identity:

In [57]:
from math import log

R = rkt.universalGasConstant
T = props.temperature()
P = props.pressure()

uCO2aq = props.speciesChemicalPotential("CO2")
u0CO2g = db.species().get("CO2(g)").props(T, P).G0
uCO2g  = u0CO2g + R*T*log(fCO2g)

print(f"μCO2(aq) = {uCO2aq} J/mol")
print(f"μCO2(g)  = {uCO2g} J/mol")

μCO2(aq) = -118583 J/mol
μCO2(g)  = -118583 J/mol


Note that both chemical potential values above are identical, meaning that the
fugacity constraint was satisfied.

With this, we conclude this tutorial, which demonstrates how Reaktoro can be
used to solve chemical equilibrium problems with constrained gas fugacity,
followed by a presentation of how to verify if the computed equilibrium state
reflects the desired fugacity value.